In [24]:
# Imports
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Nucleo Mundial de
[nltk_data]     Ne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [97]:
# Carregamento dos dados
df = pd.read_parquet('dados/train.parquet')
df = df[['user_input', 'uf', 'razaosocial', 'nome_fantasia']].reset_index(drop=True)

def clean_text(text):
    """Pré-processamento melhorado"""
    text = str(text).lower().strip()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # Preserva números e caracteres especiais importantes
    text = re.sub(r'[^a-z0-9\s&.-]', ' ', text)
    
    # Trata números como tokens especiais
    text = re.sub(r'(\d+)', r' \1 ', text)
    
    # Remove stopwords específicas do domínio
    stopwords = ['ltda', 'me', 'epp', 'sa', 's/a', 'limitada', 'eireli', 'comercio', 'servicos']
    words = text.split()
    words = [w for w in words if w not in stopwords]
    
    # Normaliza espaços
    return ' '.join(words)



In [90]:
# Criação do campo de texto e do target
df['target_empresa'] = (df.razaosocial.fillna('') + ' ' + df.nome_fantasia.fillna('')).apply(clean_text)


In [77]:
df_reduzido = df[:10000]

In [91]:
# Separação treino/teste
df_train, df_test = train_test_split(df_reduzido, 
                                     test_size=0.2, 
                                     random_state=42)


# FastText

In [ ]:

def prepare_search_text(row):
    razao = clean_text(str(row.razaosocial)) if pd.notna(row.razaosocial) else ''
    fantasia = clean_text(str(row.nome_fantasia)) if pd.notna(row.nome_fantasia) else ''
    
    if razao and fantasia:        
        return f"{fantasia} {fantasia} {razao}"
    
    return fantasia or razao

df_train['search_text'] = df_train.apply(prepare_search_text, axis=1)
df_test['search_text'] = df_test.apply(prepare_search_text, axis=1)

def tokenize_business(text):
    text = str(text).lower().strip()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    text = re.sub(r'[^a-z0-9\s&.-]', ' ', text) 
    
    text = re.sub(r'(\d+)', r' \1 ', text)
    
    text = re.sub(r'\s+', ' ', text)
    
    tokens = []
    for word in text.split():
        
        if any(c.isdigit() for c in word):
            tokens.append(word)
        else:
            if len(word) <= 2:
                tokens.append(word)
            else:
                tokens.extend(word_tokenize(word))
    return tokens


corpus_ft = [tokenize_business(text) for text in df_train.search_text]

fasttext_model = FastText(
    sentences=corpus_ft,
    vector_size=500, 
    window=5,  
    min_count=1, 
    sg=1, 
    hs=1,    
    negative=15, 
    epochs=50,  
    min_n=1,   
    max_n=8,  
    workers=4    
)



def improved_ft_embedding(text):
    
    tokens = tokenize_business(text)
    vectors = []
    weights = []
    
    for i, token in enumerate(tokens):
        if token in fasttext_model.wv:
            vec = fasttext_model.wv[token]
          
            weight = 1.0
                  
            if i < 2:
                weight *= 1.6
                      
            if len(token) <= 2:
                weight *= 0.4
                
            vectors.append(vec)
            weights.append(weight)
    
    if vectors:
        weights = np.array(weights).reshape(-1, 1)
        weighted_vectors = np.array(vectors) * weights
        
        emb = np.sum(weighted_vectors, axis=0) / np.sum(weights)
        return emb / np.linalg.norm(emb)
    
    return np.zeros(fasttext_model.vector_size)


print("Gerando embeddings FastText...")
df_train['ft_emb'] = df_train.search_text.apply(improved_ft_embedding)
df_test['ft_emb'] = df_test.search_text.apply(improved_ft_embedding)


Gerando embeddings FastText...


# TF-IDF (n-grams de caracteres)

In [ ]:

def prepare_search_text(row):
    razao = clean_text(str(row.razaosocial)) if pd.notna(row.razaosocial) else ''
    fantasia = clean_text(str(row.nome_fantasia)) if pd.notna(row.nome_fantasia) else ''
    
    
    if razao and fantasia:    
        return f"{fantasia} {fantasia} {razao}"
   
    return fantasia or razao

df_train['search_text'] = df_train.apply(prepare_search_text, axis=1)

tfidf = TfidfVectorizer(
    analyzer='char_wb', 
    ngram_range=(2, 4),
    max_df=0.95,    
    min_df=2,      
    binary=True    
)

tfidf_matrix = tfidf.fit_transform(df_train.search_text)


# BERT

In [ ]:
bert_model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')

def augment_company_name(text):
    variations = [text]    
   
    common_suffixes = ['ltda', 'limitada', 'sa', 's.a', 'sociedade anonima', 'cia', 'companhia', 'me', 'epp']
    text_clean = ' '.join([w for w in text.split() if w.lower() not in common_suffixes])
    if text_clean:
        variations.append(text_clean)    
   
    text_with_dots = re.sub(r'\s+', '. ', text_clean)
    variations.append(text_with_dots)
    
    return variations

print("Gerando embeddings BERT...")

print("Gerando embeddings para base de treino...")
train_texts = df_train.search_text.tolist()
train_embeddings = []

batch_size = 32
for i in tqdm(range(0, len(train_texts), batch_size)):
    batch = train_texts[i:i+batch_size]
    batch_embeddings = bert_model.encode(batch)
    train_embeddings.extend(batch_embeddings)

train_embeddings = np.array(train_embeddings)
df_train['bert_emb'] = list(train_embeddings)

print("\nGerando embeddings para base de teste...")
test_texts = df_test.search_text.tolist()
test_embeddings = []

for i in tqdm(range(0, len(test_texts), batch_size)):
    batch = test_texts[i:i+batch_size]
    batch_embeddings = bert_model.encode(batch)
    test_embeddings.extend(batch_embeddings)

test_embeddings = np.array(test_embeddings)
df_test['bert_emb'] = list(test_embeddings)


No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


Gerando embeddings BERT...
Gerando embeddings para base de treino...


  0%|          | 0/6387 [00:00<?, ?it/s]


Gerando embeddings para base de teste...


  0%|          | 0/1597 [00:00<?, ?it/s]

# Top-k

Essa função:

- Recebe o nome do modelo (fasttext, tfidf, bert), o texto digitado (user_input), e a base onde buscar (df_train).

- Filtra por estado (uf) se fornecido.

- Converte o user_input em vetor usando o modelo apropriado.

- Calcula similaridade de cosseno entre o vetor do input e os vetores do corpus.

- Retorna os k registros mais similares do corpus (top-k).

In [ ]:
def get_topk(model_name, user_input, uf=None, k=5, base_busca=None):
    texto = clean_text(user_input)
    data = base_busca.copy()
    
    if uf:
        data = data[data.uf == uf].reset_index(drop=True)
    
    if model_name == 'fasttext':
        emb = improved_ft_embedding(texto)
        sims = cosine_similarity([emb], list(data.ft_emb.values))[0]
    elif model_name == 'tfidf':
        vec = tfidf.transform([texto])
        indices_originais = data.index.values
        sims = cosine_similarity(vec, tfidf_matrix[indices_originais])[0]
    elif model_name == 'bert':
        emb = bert_model.encode([texto])[0]
        sims = cosine_similarity([emb], list(data.bert_emb.values))[0]

    top_indices = sims.argsort()[::-1][:k]
    return data.iloc[top_indices]


Essa função:

- Percorre todas as linhas de df_test, onde temos user_input e target_empresa.

- Usa o modelo para buscar os top-5 resultados mais parecidos com o user_input.

- Compara com o target_empresa:

- Se o alvo está em posição 0: conta como acerto Top-1

- Se o alvo está entre os 5 primeiros: conta como acerto Top-5

- Mostra o percentual de acertos para cada modelo.

In [ ]:
def avaliar_modelo(model_name, df_teste, base_busca, batch_size=32):
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    with tqdm(total=total, desc=f'Avaliando {model_name}') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                resultados = get_topk(model_name, entrada, uf, k=5, base_busca=base_busca)
                pred_empresas = resultados.target_empresa.apply(clean_text).values
                
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                
                pbar.update(1)
    
    print(f'\nModelo: {model_name}')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')


In [7]:
df_test_test = df_test[:1000]

In [93]:
avaliar_modelo('fasttext', df_test, df_train)
# avaliar_modelo('tfidf', df_test_test, df_train)
# avaliar_modelo('bert', df_test_test, df_train)


Avaliando fasttext: 100%|██████████| 2000/2000 [00:13<00:00, 149.89it/s]



Modelo: fasttext
Top-1: 49.00% | Top-5: 56.35%


In [54]:
get_topk('fasttext', 'rener', uf='SP', base_busca=df_test)

,user_input,uf,razaosocial,nome_fantasia,target_empresa,search_text,ft_emb
150,LOJA DE RENNER,SP,LOJAS RENNER S.A.,RENNER,lojas renner renner,renner renner lojas renner,"[-0.0255066041671077, -0.016622557808581004, 0..."
396,LOJAS RENNER,SP,LOJAS RENNER S.A.,RENNER,lojas renner renner,renner renner lojas renner,"[-0.0255066041671077, -0.016622557808581004, 0..."
277,RENATA,SP,PANIFICADORA RENATA LTDA,RENATA,panificadora renata renata,renata renata panificadora renata,"[-0.025602357422335697, 0.00868732782211114, 0..."
459,SOLUÇÃO DE SPRAY,SP,SPRAY MASTER LTDA,SPRAY SOLUTION,spray master spray solution,spray solution spray solution spray master,"[-0.03604829337975654, 0.05073838297492538, 0...."
591,IGREJA CRISTÃ RENASCER,SP,IGREJA CRISTA APOSTOLICA RENASCER EM CRISTO,RENASCER,igreja crista apostolica renascer em cristo re...,renascer renascer igreja crista apostolica ren...,"[-0.014231311430206907, 5.956817042287924e-05,..."


# Ensemble

In [ ]:
def get_topk_ensemble(user_input, uf=None, k=5, base_busca=None, weights={'fasttext': 0.7, 'bert': 0.4, 'tfidf': 0.1}):
    texto = clean_text(user_input)
    data = base_busca.copy()
    
    if uf:
        data = data[data.uf == uf].reset_index(drop=True)
    
    sims_dict = {}
    
    # FastText
    emb_ft = improved_ft_embedding(texto)
    sims_dict['fasttext'] = cosine_similarity([emb_ft], list(data.ft_emb.values))[0]
    
    # BERT
    emb_bert = bert_model.encode([texto])[0]
    sims_dict['bert'] = cosine_similarity([emb_bert], list(data.bert_emb.values))[0]
    
    # TF-IDF
    vec_tfidf = tfidf.transform([texto])
    indices_originais = data.index.values
    sims_dict['tfidf'] = cosine_similarity(vec_tfidf, tfidf_matrix[indices_originais])[0]
    
    # Combina as similaridades com pesos
    weighted_sims = np.zeros_like(sims_dict['fasttext'])
    for model_name, weight in weights.items():
        weighted_sims += weight * sims_dict[model_name]
    
    top_indices = weighted_sims.argsort()[::-1][:k]
    return data.iloc[top_indices]

# Função para avaliar o modelo ensemble
def avaliar_modelo_ensemble(df_teste, base_busca, batch_size=32):
    print("\nAvaliando modelo Ensemble...")
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    # Processamento em lotes com barra de progresso
    with tqdm(total=total, desc='Avaliando Ensemble') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                resultados = get_topk_ensemble(entrada, uf, k=5, base_busca=base_busca)
                pred_empresas = resultados.target_empresa.apply(clean_text).values
                
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                
                pbar.update(1)
    
    print(f'\nModelo: Ensemble')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')


In [56]:
# Para avaliar o modelo ensemble
avaliar_modelo_ensemble(df_test, df_train)


Avaliando modelo Ensemble...


Avaliando Ensemble:   0%|          | 0/2000 [00:00<?, ?it/s]


NameError: name 'bert_model' is not defined

In [51]:
get_topk_ensemble('extra', uf='SP', base_busca=df_test)

,user_input,uf,razaosocial,nome_fantasia,target_empresa,search_text,ft_emb,bert_emb
6682,LUCITERRA COMERCIO,SP,COMERCIO E EXTRACAO LUCIANO LTDA,LUCITERRA,comercio e extracao luciano luciterra,luciterra luciterra comercio e extracao luciano,"[0.06596463981014179, 0.05320055896099502, -0....","[-0.12613136, -0.26669115, 0.3316577, 0.080645..."
15969,LUSITERRA C E E,SP,COMERCIO E EXTRACAO LUCIANO LTDA,LUCITERRA,comercio e extracao luciano luciterra,luciterra luciterra comercio e extracao luciano,"[0.06596463981014179, 0.05320055896099502, -0....","[-0.12613136, -0.26669115, 0.3316577, 0.080645..."
9695,LUSITERRA C,SP,COMERCIO E EXTRACAO LUCIANO LTDA,LUCITERRA,comercio e extracao luciano luciterra,luciterra luciterra comercio e extracao luciano,"[0.06596463981014179, 0.05320055896099502, -0....","[-0.12613136, -0.26669115, 0.3316577, 0.080645..."
5394,LUCITERRA C,SP,COMERCIO E EXTRACAO LUCIANO LTDA,LUCITERRA,comercio e extracao luciano luciterra,luciterra luciterra comercio e extracao luciano,"[0.06596463981014179, 0.05320055896099502, -0....","[-0.12613136, -0.26669115, 0.3316577, 0.080645..."
3090,LUSITERRA EXTRACAO,SP,COMERCIO E EXTRACAO LUCIANO LTDA,LUCITERRA,comercio e extracao luciano luciterra,luciterra luciterra comercio e extracao luciano,"[0.06596463981014179, 0.05320055896099502, -0....","[-0.12613136, -0.26669115, 0.3316577, 0.080645..."


# Mudando pesos das distâncias

In [ ]:
# Função para calcular similaridade híbrida
def hybrid_similarity(query, target, query_vec, target_vec):
    # Similaridade de sequência de caracteres 
    char_sim = fuzz.partial_ratio(query, target) / 100.0
    
    # Similaridade semântica 
    semantic_sim = cosine_similarity([query_vec], [target_vec])[0][0]
    
    # Combina as similaridades com pesos
    # Damos mais peso para similaridade de caracteres em strings curtas
    # e mais peso para similaridade semântica em strings longas
    if len(query) < 5:  # query muito curta
        return 0.7 * char_sim + 0.3 * semantic_sim
    elif len(query) < 10:  # query média
        return 0.5 * char_sim + 0.5 * semantic_sim
    else:  # query longa
        return 0.3 * char_sim + 0.7 * semantic_sim

def get_topk_improved(user_input, uf=None, k=5, base_busca=None):
    texto = clean_text(user_input)
    data = base_busca.copy()
    
    if uf:
        data = data[data.uf == uf].reset_index(drop=True)
    
    # Gera embedding para a query
    query_ft = improved_ft_embedding(texto)
    
    # Calcula similaridades híbridas
    similarities = []
    for idx, row in data.iterrows():
        # Usa nome fantasia se disponível, senão usa razão social
        target = clean_text(row.nome_fantasia if pd.notna(row.nome_fantasia) else row.razaosocial)
        sim = hybrid_similarity(texto, target, query_ft, row.ft_emb)
        similarities.append(sim)
    
    # Retorna os top-k mais similares
    top_indices = np.array(similarities).argsort()[::-1][:k]
    return data.iloc[top_indices]

def avaliar_modelo_improved(df_teste, base_busca, batch_size=32):
    print("\nAvaliando modelo Híbrido...")
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    with tqdm(total=total, desc='Avaliando Híbrido') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                resultados = get_topk_improved(entrada, uf, k=5, base_busca=base_busca)
                pred_empresas = resultados.target_empresa.apply(clean_text).values
                
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                
                pbar.update(1)
    
    print(f'\nModelo: Híbrido')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')


In [60]:
avaliar_modelo_improved(df_test_test, df_train)

KeyboardInterrupt: 

In [ ]:
resultado = get_topk_ensemble('banco', uf='SP', k=5, base_busca=df_test)
print(resultado[['razaosocial', 'nome_fantasia']])

                         razaosocial           nome_fantasia
4791   BANCO SANTANDER (BRASIL) S.A.   AG. BOICUCANGA-INT-SP
15467  BANCO SANTANDER (BRASIL) S.A.   AG. BOICUCANGA-INT-SP
14773  BANCO SANTANDER (BRASIL) S.A.   AG. BOICUCANGA-INT-SP
2413   BANCO SANTANDER (BRASIL) S.A.  BANCO DIRETO SPAULO SP
3153             BANCO BRADESCO S.A.  BRADESCO AG GUARARAPES


In [58]:
# Testando modelo MPNet que é otimizado para similaridade semântica
print("Carregando modelo MPNet...")
mpnet_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def evaluate_model_performance(model_name, model, df_test, df_train, sample_size=2000):
    print(f"\nAvaliando modelo: {model_name}")
    
    # Prepara os textos
    train_texts = df_train.apply(lambda x: f"{x['razaosocial']} - {x['nome_fantasia']}", axis=1).tolist()[:sample_size]
    test_texts = df_test.user_input.tolist()[:sample_size]
    target_texts = df_test.apply(lambda x: f"{x['razaosocial']} - {x['nome_fantasia']}", axis=1).tolist()[:sample_size]
    
    # Gera embeddings baseado no tipo de modelo
    print("Gerando embeddings para base de treino...")
    if isinstance(model, SentenceTransformer):
        train_embeddings = model.encode(train_texts, show_progress_bar=True, batch_size=32)
        test_embeddings = model.encode(test_texts, show_progress_bar=True, batch_size=32)
    else:  # FastText
        train_embeddings = [improved_ft_embedding(text) for text in tqdm(train_texts)]
        test_embeddings = [improved_ft_embedding(text) for text in tqdm(test_texts)]
        train_embeddings = np.array(train_embeddings)
        test_embeddings = np.array(test_embeddings)
    
    # Calcula similaridades e predictions
    similarities = cosine_similarity(test_embeddings, train_embeddings)
    predictions = [train_texts[idx] for idx in similarities.argmax(axis=1)]
    
    # Calcula métricas
    correct = [pred == target for pred, target in zip(predictions, target_texts)]
    accuracy = sum(correct) / len(correct)
    
    print(f"Top-1 Accuracy: {accuracy:.2%}")
    return accuracy

# Avalia os dois modelos
print("Avaliando modelo FastText...")
original_accuracy = evaluate_model_performance("FastText", fasttext_model, df_test_test, df_train)
print("\nAvaliando modelo MPNet...")
mpnet_accuracy = evaluate_model_performance("MPNet", mpnet_model, df_test_test, df_train)

Carregando modelo MPNet...
Avaliando modelo FastText...

Avaliando modelo: FastText
Gerando embeddings para base de treino...


100%|██████████| 1000/1000 [00:00<00:00, 6372.14it/s]


Top-1 Accuracy: 34.10%

Avaliando modelo MPNet...

Avaliando modelo: MPNet
Gerando embeddings para base de treino...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Top-1 Accuracy: 15.90%


In [ ]:
# Funções de pré-processamento e cálculo de similaridade aprimoradas
import numpy as np
from fuzzywuzzy import fuzz
from sklearn.preprocessing import normalize

def preprocess_company_query(query, uf=None):
    """Pré-processa a query do usuário considerando padrões comuns"""
    query = clean_text(query)
    
    # Remove palavras muito comuns em nomes de empresa que podem atrapalhar
    common_words = ['comercio', 'industria', 'servicos', 'produtos', 'brasil', 
                   'comercial', 'distribuidora', 'representacoes']
    words = query.split()
    words = [w for w in words if w not in common_words]
    
    # Se a query for muito curta (possível abreviação), trata como prefixo
    if len(query) <= 5:
        return ' '.join(words), 'prefix'
    
    return ' '.join(words), 'full'

def advanced_similarity(query, target_razao, target_fantasia, query_emb, target_emb, uf_match=False):
    """Calcula similaridade considerando múltiplos fatores"""
    # Pré-processa query e alvos
    clean_query, query_type = preprocess_company_query(query)
    clean_razao = clean_text(target_razao)
    clean_fantasia = clean_text(target_fantasia)
    
    # Similaridade semântica 
    semantic_sim = cosine_similarity([query_emb], [target_emb])[0][0]
    
    # Similaridade de caracteres
    char_sim_razao = fuzz.partial_ratio(clean_query, clean_razao) / 100.0
    char_sim_fantasia = fuzz.partial_ratio(clean_query, clean_fantasia) / 100.0
    
    # Pega a melhor similaridade entre razão social e nome fantasia
    char_sim = max(char_sim_razao, char_sim_fantasia)
    
    # Prefixo exato (para queries curtas)
    if query_type == 'prefix':
        if clean_fantasia.startswith(clean_query):
            char_sim = 1.0
        elif clean_razao.startswith(clean_query):
            char_sim = 0.9
    
    # Ajusta pesos baseado no tipo de query e match de UF
    if query_type == 'prefix':
        # Para queries curtas, prioriza match exato de caracteres
        final_sim = 0.7 * char_sim + 0.3 * semantic_sim
    else:
        # Para queries longas, equilibra entre char e semantic
        final_sim = 0.5 * char_sim + 0.5 * semantic_sim
    
    # Boost para match de UF
    if uf_match:
        final_sim *= 1.2
    
    return final_sim

def get_topk_advanced(user_input, uf=None, k=5, base_busca=None):
    """Versão aprimorada do get_topk usando apenas FastText com pesos dinâmicos"""
    texto = clean_text(user_input)
    data = base_busca.copy()
    
    # Primeiro filtra por UF para reduzir o espaço de busca
    if uf:
        data = data[data.uf == uf]
    
    # Se não houver dados após o filtro, retorna DataFrame vazio
    if len(data) == 0:
        return pd.DataFrame(columns=data.columns)
    
    # Gera embedding usando FastText
    query_ft = improved_ft_embedding(texto)
    
    # Calcula similaridades
    similarities = []
    for idx, row in data.iterrows():
        # Clean query e targets
        clean_query = clean_text(texto)
        clean_razao = clean_text(str(row.razaosocial))
        clean_fantasia = clean_text(str(row.nome_fantasia))
        
        # Similaridade FastText
        ft_sim = cosine_similarity([query_ft], [row.ft_emb])[0][0]
        
        # Similaridade de caracteres (usando fuzzy matching)
        char_sim_razao = fuzz.partial_ratio(clean_query, clean_razao) / 100.0
        char_sim_fantasia = fuzz.partial_ratio(clean_query, clean_fantasia) / 100.0
        char_sim = max(char_sim_razao, char_sim_fantasia)
        
        # Pesos dinâmicos baseados no tamanho da query
        if len(clean_query) <= 5:  # Query curta
            final_sim = 0.3 * ft_sim + 0.7 * char_sim
        else:  # Query longa
            final_sim = 0.7 * ft_sim + 0.3 * char_sim
        
        # Boost para match de UF
        if uf and uf == row.uf:
            final_sim *= 1.2
        
        similarities.append(final_sim)
    
    # Retorna os top-k mais similares
    similarities = np.array(similarities)
    if len(similarities) > 0:
        top_indices = similarities.argsort()[::-1][:k]
        return data.iloc[top_indices]
    else:
        return pd.DataFrame(columns=data.columns)

def avaliar_modelo_advanced(df_teste, base_busca, batch_size=32):
    """Avaliação usando o modelo avançado"""
    print("\nAvaliando modelo Avançado...")
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    with tqdm(total=total, desc='Avaliando modelo Avançado') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                resultados = get_topk_advanced(entrada, uf, k=5, base_busca=base_busca)
                if len(resultados) > 0:
                    pred_empresas = resultados.target_empresa.apply(clean_text).values
                    
                    if alvos[i+j] == pred_empresas[0]:
                        top1 += 1
                    if alvos[i+j] in pred_empresas:
                        top5 += 1
                
                pbar.update(1)
    
    print(f'\nModelo: Avançado')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')


In [60]:
avaliar_modelo_advanced(df_teste=df_test, base_busca=df_train)


Avaliando modelo Avançado...


Avaliando modelo Avançado: 100%|██████████| 2000/2000 [11:37<00:00,  2.87it/s]


Modelo: Avançado
Top-1: 50.45% | Top-5: 57.60%


# Avaliando diferentes distâncias com fasttext

In [ ]:
# Avaliação de diferentes métricas de similaridade para matching de empresas
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cdist
from tqdm import tqdm
import Levenshtein
from joblib import Parallel, delayed

# Pré-computar todos os vetores e normalizá-los uma única vez
print("Preparando vetores...")
fasttext_matrix = np.vstack(df_train['fasttext_vec'].values)
fasttext_matrix = normalize(fasttext_matrix)

# Processar em lotes para reduzir uso de memória
BATCH_SIZE = 100

def process_batch(batch_entries, target_vecs, metric='cosine'):
    results = []
    
    for entrada, alvo in batch_entries:
        # Preparar vetor do input
        user_vec = improved_ft_embedding(entrada)
        user_vec = normalize(user_vec.reshape(1, -1))
        
        # Calcular similaridades de uma vez
        if metric == 'cosine':
            sims = cosine_similarity(user_vec, target_vecs)[0]
        elif metric == 'euclidean':
            sims = -euclidean_distances(user_vec, target_vecs)[0]
        elif metric == 'manhattan':
            sims = -manhattan_distances(user_vec, target_vecs)[0]
        elif metric == 'jaccard':
            input_tokens = set(tokenize_business(entrada))
            target_tokens = [set(tokenize_business(text)) for text in df_train['search_text']]
            sims = np.array([len(input_tokens & tokens) / len(input_tokens | tokens) if tokens else 0 
                           for tokens in target_tokens])
        elif metric == 'levenshtein':
            clean_input = clean_text(entrada)
            sims = np.array([-Levenshtein.distance(clean_input, target) 
                           for target in df_train['target_empresa']])
        
        # Encontrar top-k mais similares
        top_indices = np.argsort(sims)[::-1][:5]
        ranked_empresas = df_train.iloc[top_indices]['target_empresa'].values
        
        # Verificar acertos
        results.append((alvo in ranked_empresas[0:1], alvo in ranked_empresas))
    
    return results

def avaliar_metricas_topk_otimizado(df_teste, metricas=['cosine', 'euclidean']):
    resultados = {m: {'top@1': 0, 'top@5': 0} for m in metricas}
    total_samples = len(df_teste)
    
    # Preparar dados de entrada
    entries = list(zip(df_teste['user_input'], df_teste['target_empresa']))
    
    # Processar cada métrica
    for metric in metricas:
        print(f"\nProcessando métrica: {metric}")
        
        # Processar em lotes paralelos
        batches = [entries[i:i+BATCH_SIZE] for i in range(0, len(entries), BATCH_SIZE)]
        
        with Parallel(n_jobs=-1) as parallel:
            results = []
            for batch_results in tqdm(
                parallel(delayed(process_batch)(batch, fasttext_matrix, metric) 
                        for batch in batches),
                total=len(batches)
            ):
                results.extend(batch_results)
        
        # Agregar resultados
        top1 = sum(r[0] for r in results)
        top5 = sum(r[1] for r in results)
        
        resultados[metric]['top@1'] = top1 / total_samples
        resultados[metric]['top@5'] = top5 / total_samples
    
    return pd.DataFrame(resultados)

# Executar avaliação otimizada
print("Iniciando avaliação...")
resultado_metricas = avaliar_metricas_topk_otimizado(
    df_test_test, 
    metricas=['cosine', 'euclidean', 'manhattan', 'jaccard', 'levenshtein']
)
print("\nResultados:")
print(resultado_metricas.T)

Preparando vetores...
Iniciando avaliação...

Processando métrica: cosine


c:\Users\Nucleo Mundial de Ne\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 10/10 [00:00<?, ?it/s]



Processando métrica: euclidean


100%|██████████| 10/10 [00:00<00:00, 1309.12it/s]



Processando métrica: manhattan


100%|██████████| 10/10 [00:00<00:00, 4993.22it/s]



Processando métrica: jaccard


# Testes com char-CNN e Autoencoders

In [ ]:
# Testes com char-CNN e Autoencoders otimizados para matching de empresas (PyTorch CPU)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import numpy as np

# Configurando device para usar CPU (GPU com AMD não está rolando)
device = torch.device('cpu')
print(f"Usando device: {device}")

# Configurações otimizadas para nomes de empresas
MAX_LEN = 50
CHAR_EMBED_DIM = 32
BATCH_SIZE = 128
N_EPOCHS = 5

# Preparação dos dados e modelo
print("Preparando dados...")
empresa_texts = df_train.apply(lambda x: f"{x['nome_fantasia']} {x['razaosocial']}", axis=1).values

# Tokenização a nível de caractere
char_tokenizer = Tokenizer(char_level=True, 
                         filters='!"#$%&()*+/<=>@[\]^_`{|}~\t\n',
                         lower=True, 
                         oov_token='<UNK>')
char_tokenizer.fit_on_texts(empresa_texts)

# Converter textos para sequências e fazer padding
sequences = char_tokenizer.texts_to_sequences(empresa_texts)
padded_seqs = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

# Split dados
X_train_char, X_val_char = train_test_split(padded_seqs, test_size=0.1, random_state=42)

# Criar datasets e dataloaders
train_dataset = CompanyNameDataset(X_train_char)
val_dataset = CompanyNameDataset(X_val_char)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Inicializar modelo
print("\nConstruindo modelo char-CNN...")
vocab_size = len(char_tokenizer.word_index) + 1
model = CharCNN(vocab_size, CHAR_EMBED_DIM, MAX_LEN).to(device)

# Otimizador e função de perda
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

# Treinamento
print("\nTreinando modelo...")
best_val_loss = float('inf')
patience = 3
patience_counter = 0

for epoch in range(N_EPOCHS):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        encoded = model(batch)  # [batch_size, 64]
        
        # Target é o próprio batch após embedding com redução por média
        target = model.embedding(batch)  # [batch_size, seq_len, embed_dim]
        target = target.mean(dim=1)  # [batch_size, embed_dim]
        
        # Garante que as dimensões são compatíveis
        if target.size(1) != encoded.size(1):
            target = F.linear(target, model.encoder.weight.t())  # Projeta para mesma dimensão do encoded
        
        # Calcula loss
        loss = criterion(encoded, target)
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    train_loss = total_loss / len(train_loader)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            encoded = model(batch)
            target = model.embedding(batch).mean(dim=1)
            if target.size(1) != encoded.size(1):
                target = F.linear(target, model.encoder.weight.t())
            loss = criterion(encoded, target)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}/{N_EPOCHS}:')
    print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Carregar melhor modelo
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Gerar embeddings para busca
print("\nGerando embeddings...")
all_sequences = torch.LongTensor(padded_seqs).to(device)
with torch.no_grad():
    empresa_embeddings = model(all_sequences) 
    empresa_embeddings = empresa_embeddings.cpu().numpy() 

# Preparar o KNN
print("\nPreparando índice KNN...")
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(empresa_embeddings)

# Função para buscar empresas similares
def find_similar_companies(query_text, top_k=5):
    """Busca empresas similares usando char-CNN + KNN"""
    # Preprocessa a query
    query_seq = char_tokenizer.texts_to_sequences([query_text])
    query_pad = pad_sequences(query_seq, maxlen=MAX_LEN, padding='post')
    query_tensor = torch.LongTensor(query_pad).to(device)
    
    # Gera embedding
    with torch.no_grad():
        query_emb = model(query_tensor).cpu().numpy()
    
    # Busca vizinhos mais próximos
    distances, indices = knn.kneighbors(query_emb)
    
    # Retorna resultados
    results = pd.DataFrame({
        'empresa': empresa_texts[indices[0]],
        'similaridade': 1 - distances[0]
    })
    
    return results

# Avaliação do char-CNN
def avaliar_char_cnn(df_teste, batch_size=32):
    """Avalia o modelo char-CNN"""
    print("\nAvaliando modelo char-CNN...")
    model.eval()
    total = len(df_teste)
    top1 = 0
    top5 = 0
    
    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values
    
    with tqdm(total=total, desc='Avaliando char-CNN') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                # Preprocessa a query
                query_seq = char_tokenizer.texts_to_sequences([entrada])
                query_pad = pad_sequences(query_seq, maxlen=MAX_LEN, padding='post')
                query_tensor = torch.LongTensor(query_pad).to(device)
                
                # Gera embedding
                with torch.no_grad():
                    encoded = model(query_tensor)  
                    query_emb = encoded.cpu().numpy()
                
                # Busca vizinhos mais próximos
                distances, indices = knn.kneighbors(query_emb)
                
                # Prepara resultados
                pred_empresas = empresa_texts[indices[0]]
                
                # Limpa predições para comparação
                pred_empresas_clean = [clean_text(emp) for emp in pred_empresas]
                
                if alvos[i+j] == pred_empresas_clean[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                
                pbar.update(1)
    
    print(f'\nModelo: char-CNN')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')


Usando device: cpu
Preparando dados...

Construindo modelo char-CNN...

Treinando modelo...

Construindo modelo char-CNN...

Treinando modelo...


AttributeError: 'tuple' object has no attribute 'size'

# Fasttext e knn

In [73]:
# Função para avaliar métricas Top-1 e Top-5
def evaluate_topk_metrics(knn_model, df_test, df_train, metric='cosine', batch_size=32):
    """
    Avalia as métricas Top-1 e Top-5 para o modelo KNN com FastText
    """
    total = len(df_test)
    top1_hits = 0
    top5_hits = 0
    
    print(f"\nAvaliando com métrica: {metric}")
    
    # Processa em batches para melhor performance e memória
    with tqdm(total=total, desc=f'Avaliando {metric}') as pbar:
        for start_idx in range(0, total, batch_size):
            end_idx = min(start_idx + batch_size, total)
            batch = df_test.iloc[start_idx:end_idx]
            
            for _, row in batch.iterrows():
                try:
                    # Gera embedding para o input
                    query_text = clean_text(row['user_input'])
                    query_vec = get_text_vector(query_text, model_ft).reshape(1, -1)
                    
                    # Busca os k vizinhos mais próximos
                    distances, indices = knn_model.kneighbors(query_vec)
                    
                    # Obtém as empresas preditas
                    predicted = df_train.iloc[indices[0]]['target_empresa'].values
                    true_company = clean_text(row['target_empresa'])
                    
                    # Verifica Top-1
                    if clean_text(predicted[0]) == true_company:
                        top1_hits += 1
                    
                    # Verifica Top-5
                    if true_company in [clean_text(p) for p in predicted]:
                        top5_hits += 1
                    
                except Exception as e:
                    print(f"Erro ao processar entrada: {str(e)}")
                    continue
                
                pbar.update(1)
    
    # Calcula métricas finais
    top1_accuracy = top1_hits / total
    top5_accuracy = top5_hits / total
    
    print(f"\nResultados para {metric}:")
    print(f"Top-1 Accuracy: {top1_accuracy:.4f} ({top1_hits}/{total})")
    print(f"Top-5 Accuracy: {top5_accuracy:.4f} ({top5_hits}/{total})")
    
    return {
        'metric': metric,
        'top1_accuracy': top1_accuracy,
        'top5_accuracy': top5_accuracy,
        'top1_hits': top1_hits,
        'top5_hits': top5_hits,
        'total': total
    }

# Treinar KNN com diferentes métricas
print("Treinando modelos KNN...")
knn_models = {
    'cosine': NearestNeighbors(n_neighbors=5, metric='cosine', n_jobs=-1),
    'euclidean': NearestNeighbors(n_neighbors=5, metric='euclidean', n_jobs=-1),
    'manhattan': NearestNeighbors(n_neighbors=5, metric='manhattan', n_jobs=-1)
}

# Gerar embeddings para o conjunto de treino
print("Gerando embeddings para treino...")
train_vectors = np.vstack([
    get_text_vector(text, model_ft) 
    for text in tqdm(df_train['target_empresa'].apply(clean_text))
])

# Treinar cada modelo KNN
for metric, model in knn_models.items():
    print(f"\nTreinando KNN com {metric}...")
    model.fit(train_vectors)

# Avaliar cada métrica
results = []
for metric in ['cosine', 'euclidean', 'manhattan']:
    result = evaluate_topk_metrics(
        knn_models[metric],
        df_test,
        df_train,
        metric=metric
    )
    results.append(result)

# Mostrar resultados comparativos
print("\nComparativo de métricas:")
df_results = pd.DataFrame(results)
print(df_results.set_index('metric')[['top1_accuracy', 'top5_accuracy']])

# Exemplo de uso
def get_similar_companies(query_text, metric='cosine', k=5):
    """Retorna as k empresas mais similares"""
    try:
        # Preprocessa e gera embedding
        query_clean = clean_text(query_text)
        query_vec = get_text_vector(query_clean, model_ft).reshape(1, -1)
        
        # Busca vizinhos mais próximos
        distances, indices = knn_models[metric].kneighbors(query_vec)
        
        # Prepara resultados
        results = pd.DataFrame({
            'empresa': df_train.iloc[indices[0]]['target_empresa'].values,
            'distancia': distances[0],
            'score': 1 - distances[0]  # Converte distância em score
        })
        
        return results.sort_values('score', ascending=False)
    
    except Exception as e:
        print(f"Erro ao buscar empresas similares: {str(e)}")
        return pd.DataFrame(columns=['empresa', 'distancia', 'score'])

# Teste com algumas queries
test_queries = ["banco itau", "carrefour", "casas bahia"]
print("\nTestando algumas queries:")
for query in test_queries:
    print(f"\nQuery: {query}")
    for metric in ['cosine', 'euclidean', 'manhattan']:
        print(f"\n{metric.capitalize()}:")
        results = get_similar_companies(query, metric=metric, k=3)
        print(results)

Treinando modelos KNN...
Gerando embeddings para treino...


100%|██████████| 8000/8000 [00:02<00:00, 3005.53it/s]



Treinando KNN com cosine...

Treinando KNN com euclidean...

Treinando KNN com manhattan...

Avaliando com métrica: cosine


Avaliando cosine: 100%|██████████| 2000/2000 [00:36<00:00, 55.41it/s]



Resultados para cosine:
Top-1 Accuracy: 0.1450 (290/2000)
Top-5 Accuracy: 0.2280 (456/2000)

Avaliando com métrica: euclidean


...

Avaliando euclidean:   0%|          | 0/2000 [00:13<?, ?it/s]

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar en

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar en

Avaliando manhattan:   0%|          | 0/2000 [00:06<?, ?it/s]

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar en

In [74]:
# Avaliar métricas
results = []
for metric in ['cosine', 'euclidean', 'manhattan']:
    result = evaluate_topk_metrics(
        knn_models[metric],
        df_test,
        df_train,
        metric=metric
    )
    results.append(result)

# Buscar empresas similares
similar = get_similar_companies("banco itau", metric='cosine', k=5)
print(similar)


Avaliando com métrica: cosine


Avaliando cosine: 100%|██████████| 2000/2000 [00:36<00:00, 54.41it/s]



Resultados para cosine:
Top-1 Accuracy: 0.1450 (290/2000)
Top-5 Accuracy: 0.2280 (456/2000)

Avaliando com métrica: euclidean


Avaliando euclidean:   0%|          | 0/2000 [00:00<?, ?it/s]

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar en

...

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'

Resultados para euclidean:
Top-1 Accuracy: 0.0000 (0/2000)
Top-5 Accuracy: 0.0000 (0/2000)

Avaliando com métrica: manhattan


Avaliando manhattan:   0%|          | 0/2000 [00:00<?, ?it/s]

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar en

Avaliando manhattan:   0%|          | 0/2000 [00:06<?, ?it/s]

Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar entrada: 'NoneType' object has no attribute 'split'
Erro ao processar en

# Novos testes

## Criar ensemble 

In [98]:
def hybrid_similarity(query, target, query_vec, target_vec, uf_match=False):
    """Similaridade híbrida com múltiplos fatores"""
    # Similaridade FastText
    ft_sim = cosine_similarity([query_vec], [target_vec])[0][0]
    
    # Similaridade de caracteres
    char_sim = fuzz.ratio(query, target) / 100.0
    
    # Similaridade de prefixo
    prefix_sim = 1.0 if target.startswith(query) else 0.0
    
    # Similaridade por tokens comuns
    query_tokens = set(query.split())
    target_tokens = set(target.split()) 
    token_sim = len(query_tokens & target_tokens) / len(query_tokens | target_tokens)
    
    # Pesos diferentes baseado no tamanho da query
    if len(query) < 5:
        final_sim = 0.4*ft_sim + 0.3*char_sim + 0.2*prefix_sim + 0.1*token_sim
    else:
        final_sim = 0.5*ft_sim + 0.2*char_sim + 0.1*prefix_sim + 0.2*token_sim
    
    # Boost para match de UF
    if uf_match:
        final_sim *= 1.2
        
    return final_sim

def avaliar_hybrid_similarity(df_teste, base_busca, batch_size=32):
    """Avalia as métricas Top-1 e Top-5 usando hybrid_similarity"""
    print("\nAvaliando modelo Híbrido...")
    total = len(df_teste)
    top1 = 0 
    top5 = 0

    # Pré-processamento dos alvos
    alvos = df_teste.target_empresa.apply(clean_text).values

    with tqdm(total=total, desc='Avaliando Híbrido') as pbar:
        for i in range(0, total, batch_size):
            batch = df_teste.iloc[i:i+batch_size]
            
            for j, row in enumerate(batch.itertuples()):
                entrada = row.user_input
                uf = row.uf if hasattr(row, 'uf') else None
                
                texto = clean_text(entrada)
                query_vec = improved_ft_embedding(texto)
                
                # Calcula similaridades
                similarities = []
                for idx, target_row in base_busca.iterrows():
                    target = clean_text(target_row.target_empresa)
                    target_vec = target_row.ft_emb
                    
                    sim = hybrid_similarity(
                        texto, 
                        target,
                        query_vec, 
                        target_vec,
                        uf_match=(uf == target_row.uf if uf else False)
                    )
                    similarities.append(sim)

                # Obtém top-k mais similares
                top_indices = np.array(similarities).argsort()[::-1][:5]
                pred_empresas = base_busca.iloc[top_indices].target_empresa.apply(clean_text).values
                
                # Atualiza métricas
                if alvos[i+j] == pred_empresas[0]:
                    top1 += 1
                if alvos[i+j] in pred_empresas:
                    top5 += 1
                    
                pbar.update(1)
    
    print(f'\nModelo: Hybrid Similarity')
    print(f'Top-1: {top1/total:.2%} | Top-5: {top5/total:.2%}')
    
    return {'top1': top1/total, 'top5': top5/total}

In [ ]:
# Exemplo de uso
resultados = avaliar_hybrid_similarity(df_test, df_train)
print(f"Top-1 Accuracy: {resultados['top1']:.4f}")
print(f"Top-5 Accuracy: {resultados['top5']:.4f}")


Avaliando modelo Híbrido...


Avaliando Híbrido: 100%|██████████| 1000/1000 [33:20<00:00,  2.00s/it]


Modelo: Hybrid Similarity
Top-1: 52.00% | Top-5: 60.00%
Top-1 Accuracy: 0.5200
Top-5 Accuracy: 0.6000


# Crossvalidation

In [109]:
df_test_test.shape

(1000, 5)

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tqdm.notebook import tqdm

def cross_validate_hybrid_similarity(df, n_splits=5, batch_size=32):
    """
    Realiza validação cruzada do modelo hybrid_similarity
    
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Arrays para guardar métricas de cada fold
    top1_scores = []
    top5_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        print(f"\nFold {fold+1}/{n_splits}")
        
        # Separa dados de treino e validação
        df_train = df.iloc[train_idx].reset_index(drop=True)
        df_val = df.iloc[val_idx].reset_index(drop=True)
        
                
        # Gera embeddings para treino e validação
        print("Gerando embeddings para o fold...")
        df_train['ft_emb'] = df_train.target_empresa.apply(improved_ft_embedding)
        df_val['ft_emb'] = df_val.target_empresa.apply(improved_ft_embedding)
        
        # Avalia o modelo neste fold
        print("Avaliando fold...")
        total = len(df_val)
        top1 = 0
        top5 = 0
        
        # Pré-processamento dos alvos
        alvos = df_val.target_empresa.apply(clean_text).values
        
        with tqdm(total=total, desc=f'Avaliando Fold {fold+1}') as pbar:
            for i in range(0, total, batch_size):
                batch = df_val.iloc[i:i+batch_size]
                
                for j, row in enumerate(batch.itertuples()):
                    entrada = row.user_input
                    uf = row.uf if hasattr(row, 'uf') else None
                    
                    texto = clean_text(entrada)
                    query_vec = improved_ft_embedding(texto)
                    
                    # Calcula similaridades híbridas
                    similarities = []
                    for idx, target_row in df_train.iterrows():
                        target = clean_text(target_row.target_empresa)
                        target_vec = target_row.ft_emb
                        
                        sim = hybrid_similarity(
                            texto,
                            target,
                            query_vec,
                            target_vec,
                            uf_match=(uf == target_row.uf if uf else False)
                        )
                        similarities.append(sim)
                    
                    # Obtém top-k mais similares
                    top_indices = np.array(similarities).argsort()[::-1][:5]
                    pred_empresas = df_train.iloc[top_indices].target_empresa.apply(clean_text).values
                    
                    # Atualiza métricas
                    if alvos[i+j] == pred_empresas[0]:
                        top1 += 1
                    if alvos[i+j] in pred_empresas:
                        top5 += 1
                    
                    pbar.update(1)
        
        # Calcula métricas do fold
        fold_top1 = top1/total
        fold_top5 = top5/total
        
        print(f'\nResultados do Fold {fold+1}:')
        print(f'Top-1: {fold_top1:.2%} | Top-5: {fold_top5:.2%}')
        
        top1_scores.append(fold_top1)
        top5_scores.append(fold_top5)
    
    # Calcula e mostra métricas médias
    print("\nResultados Finais da Validação Cruzada:")
    print(f"Top-1 Média: {np.mean(top1_scores):.4f} (±{np.std(top1_scores):.4f})")
    print(f"Top-5 Média: {np.mean(top5_scores):.4f} (±{np.std(top5_scores):.4f})")
    
    return {
        'top1_mean': np.mean(top1_scores),
        'top1_std': np.std(top1_scores),
        'top5_mean': np.mean(top5_scores),
        'top5_std': np.std(top5_scores),
        'top1_scores': top1_scores,
        'top5_scores': top5_scores
    }

# Exemplo de uso
print("Iniciando validação cruzada com modelo hybrid_similarity...")
results = cross_validate_hybrid_similarity(df_reduzido)

# Visualização detalhada dos resultados
print("\nDesempenho por fold:")
for fold, (top1, top5) in enumerate(zip(results['top1_scores'], results['top5_scores'])):
    print(f"Fold {fold+1}: Top-1: {top1:.4f} | Top-5: {top5:.4f}")

Iniciando validação cruzada com modelo hybrid_similarity...

Fold 1/5
Gerando embeddings para o fold...
Avaliando fold...


Avaliando Fold 1:   0%|          | 0/2000 [00:00<?, ?it/s]


Resultados do Fold 1:
Top-1: 48.65% | Top-5: 58.80%

Fold 2/5
Gerando embeddings para o fold...
Avaliando fold...


Avaliando Fold 2:   0%|          | 0/2000 [00:00<?, ?it/s]


Resultados do Fold 2:
Top-1: 47.25% | Top-5: 58.40%

Fold 3/5
Gerando embeddings para o fold...
Avaliando fold...


Avaliando Fold 3:   0%|          | 0/2000 [00:00<?, ?it/s]


Resultados do Fold 3:
Top-1: 49.00% | Top-5: 57.90%

Fold 4/5
Gerando embeddings para o fold...
Avaliando fold...


Avaliando Fold 4:   0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 